In [1]:
import os
import pandas as pd
import ndex2
from ndex2.cx2 import RawCX2NetworkFactory, CX2Network, NetworkXToCX2NetworkFactory
from ndex2 import NetworkXFactory
import networkx as nx
from netcoloc import netprop_zscore
from netcoloc import netprop
from netcoloc import network_colocalization
import sys
import random
#os.chdir('/tscc/projects/ps-palmer/brittany/rare_common_alcohol/rare_common_alcohol_comparison/notebooks/')
#from rca_functions import *
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/scripts')
from network_functions import *
#from network_validation_functions import *
#from plotting_functions import *
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/')

In [2]:
#create a file called environ_ndex_meta.py where you save variables 'ndex_user' and 'ndex_password'
#otherwise will prompt you to define those within the notebooks
if os.path.isfile('../environ_ndex_meta.py'):
    print ('NDEx credentials imported from meta file')
    sys.path.insert(1, '../')
    from environ_ndex_meta import *
    sys.path.pop(1)
else:
    # Prompt the user for a username
    ndex_user = input("Enter your NDEx username: ")
    # Prompt the user for a password
    ndex_password = input("Enter your NDEx password: ")

NDEx credentials imported from meta file


In [3]:
ndex_server = 'public.ndexbio.org'

In [4]:
hgnc=pd.read_csv('HGNC_protein.txt',sep='\t')
hgnc=hgnc[~hgnc['UniProt accession'].isna()]
ortho=pd.read_csv('/tscc/projects/ps-palmer/brittany/orthology_ref_tbls/ORTHOLOGY-ALLIANCE_COMBINED_2024.tsv',sep='\t',skiprows=15)
ortho=ortho[(ortho['Gene1SpeciesName']=='Rattus norvegicus')&(ortho['Gene2SpeciesName']=='Homo sapiens')]
ortho=ortho[ortho['IsBestRevScore']=='Yes']

# Interactome Set-up

pcnet2- versions 
from wright et al. 2024 preprint:
PCNet 2.0= best-performing ranked composite (top 15 interactomes, 3.85M interactions)
PCNet 2.1= top 8 interactomes, 1.75M interactions
PCNet 2.2= top 10 co-citation-free interactomes, 3.32M interactions 

In [5]:
interactome_name='signor_rat'

In [6]:
i=UUIDs['signor_rat']

In [30]:
#interactome=import_interactome(UUIDs=UUIDs,interactome_name=interactome_name)
#all_nodes=list(interactome.nodes())
#out of date- update for new version of ndex2
G=ndex2.create_nice_cx_from_server(ndex_server, username=ndex_user, password=ndex_password, uuid=i)

In [31]:
G.print_summary()

Name: Signor Complete - Rat
Nodes: 8340
Edges: 25667
Node Attributes: 17286
Edge Attributes: 256670



In [9]:
g=G.to_networkx()

In [34]:
attr_dict = {}
for node_id, attrs in G.get_nodes().items():
    node_name = attrs.get('n')  # or use attrs['@id'] if nodes are not renamed
    if node_name is None:
        continue
    node_attrs = {}
    for attr in net.get_node_attributes():
        if attr['po'] == node_id:
            node_attrs[attr['n']] = attr['v']
    attr_dict[node_name] = node_attrs

# Add attributes to NetworkX graph
nx.set_node_attributes(g, attr_dict)

AttributeError: 'dict_items' object has no attribute 'items'

In [10]:
Gfile = ndex2.create_nice_cx_from_file('signor_interactomes/SignorComplete-Rat.cx')
gfile=Gfile.to_networkx()
Gfile.print_summary()
#check the file version to the one on ndex- ndex one has more nodes and edges, so going to use that one

Name: Signor Complete - Rat
Nodes: 8202
Edges: 24033
Node Attributes: 17002
Edge Attributes: 240330



In [15]:
len(set(g.nodes).intersection(ortho.Gene1Symbol))

3391

In [16]:
len(set(gfile.nodes).intersection(ortho.Gene1Symbol))

3270

In [12]:
t=hgnc.merge(ortho, left_on='Approved symbol',right_on='Gene2Symbol')

In [17]:
g=g.subgraph(set(t.Gene1Symbol))
g=g.subgraph([node for node in g.nodes if g.degree(node) > 0])

In [21]:
len(g.nodes())

1093

In [27]:
len(g.edges())

890

In [19]:
gfile=gfile.subgraph(set(t.Gene1Symbol))
gfile=gfile.subgraph([node for node in g.nodes if g.degree(node) > 0])

In [20]:
len(gfile.nodes)

1069

In [29]:
net_cx = ndex2.create_nice_cx_from_networkx(g)
net_cx.set_name('signor_rat_protein_only')
network_uuid = net_cx.upload_to(ndex_server, ndex_user, ndex_password)

Generating CX


# read in magma genes


In [37]:
ortho_mapping=pd.Series(ortho.Gene1Symbol.values,index=ortho.Gene2Symbol).to_dict()

In [38]:
loco_magma=pd.read_csv('magma/loco_final_cf_7k_win10.genes.out',delim_whitespace=True,index_col=None,skiprows=1)

In [39]:
ext_magma=pd.read_csv('magma/seed_genes/ext_orig_annot.tsv',sep='\t')

In [40]:
loco_magma

,GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P
0,Vom2r1,1,765164,795885,1,1,7679,1.18250,0.118500
1,Vom2r5,1,1151906,1180559,2,1,7679,1.66460,0.047999
2,Vom2r6,1,1204035,1232755,2,1,7679,1.77350,0.038072
3,LOC120098769,1,1954049,1978983,5,1,7679,-0.67169,0.749110
4,Raet1l,1,1969433,1994410,1,1,7679,0.19892,0.421160
...,...,...,...,...,...,...,...,...,...
18966,Ascc3,20,53500137,53805446,347,1,7679,0.82694,0.204130
18967,Sim1,20,53817601,53917219,152,1,7679,0.88070,0.189240
18968,LOC103694460,20,54404724,54428200,56,1,7679,0.85508,0.196250
18969,Cd99,20,54408239,54432882,53,1,7679,0.86448,0.193660


In [41]:
set(g.nodes).intersection(loco_magma.GENE)

{'Abca7',
 'Abcg2',
 'Abi2',
 'Abi3',
 'Acan',
 'Acat1',
 'Ackr4',
 'Acox1',
 'Acss2',
 'Actb',
 'Ada',
 'Adam10',
 'Adamts1',
 'Adamts13',
 'Adamts4',
 'Adamts5',
 'Adgrg1',
 'Adm',
 'Adora1',
 'Adora2a',
 'Adora2b',
 'Adrb3',
 'Afp',
 'Agl',
 'Agrn',
 'Ahsa1',
 'Ahsp',
 'Akap11',
 'Akap8l',
 'Aldh1a1',
 'Aldh3a1',
 'Aldh3a2',
 'Aldh3b1',
 'Aldob',
 'Alpk1',
 'Als2',
 'Alx4',
 'Ank1',
 'Ank2',
 'Ankrd1',
 'Ankrd26',
 'Anxa13',
 'Ap1g2',
 'Aplnr',
 'Apoa1',
 'Apoa5',
 'Apoc2',
 'Apoc3',
 'Apoh',
 'Aqp2',
 'Arel1',
 'Arf1',
 'Arf3',
 'Arfgef1',
 'Arhgap18',
 'Arih1',
 'Arsa',
 'Arvcf',
 'Asap3',
 'Asb2',
 'Asf1a',
 'Asf1b',
 'Ash1l',
 'Ate1',
 'Atf5',
 'Atf6b',
 'Atg2a',
 'Atoh1',
 'Atp1b1',
 'Atp2b2',
 'Atp6v0c',
 'Atp6v1h',
 'Atp8b1',
 'Atxn3',
 'Avpr1a',
 'Bag1',
 'Bag6',
 'Bcs1l',
 'Best1',
 'Bhlhe40',
 'Bicd2',
 'Bmp1',
 'Bnip1',
 'Bpi',
 'Brap',
 'Brd2',
 'Brd3',
 'Brd7',
 'Brk1',
 'Brms1',
 'Bsg',
 'Bzw2',
 'C1qa',
 'C1qb',
 'C1qbp',
 'C1qc',
 'C3',
 'C3ar1',
 'C4b',
 'Cacna1a',


In [110]:
ext_bonf_cut=0.05/len(ext_magma)
ext_bonf_cut

2.7003672499459928e-06

In [42]:
ext_magma=ext_magma.merge(ortho[['Gene2Symbol','Gene1Symbol']],left_on='GENE',right_on='Gene2Symbol',how='left')
ext_magma

,ID,GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P,Q,Gene2Symbol,Gene1Symbol
0,105378580,LOC105378580,1,793398,815130,2,1,1492085,1.269700,0.102100,0.214091,NaN,NaN
1,148398,SAMD11,1,915741,954581,60,6,1492085,-0.920460,0.821330,0.864259,SAMD11,Samd11
2,26155,NOC2L,1,934203,969299,23,2,1492085,0.053753,0.478570,0.603460,NOC2L,Noc2l
3,339451,KLHL17,1,949952,975720,10,1,1492085,0.542700,0.293670,0.437381,KLHL17,Klhl17
4,84069,PLEKHN1,1,956497,985108,3,1,1492085,1.918900,0.027500,0.084303,PLEKHN1,Plekhn1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18859,105373099,LOC105373099,22,50596999,50620205,63,14,1492085,2.847000,0.002207,0.012196,NaN,NaN
18860,410,ARSA,22,50612754,50638173,74,12,1492085,1.106300,0.134300,0.257876,ARSA,Arsa
18861,85358,SHANK3,22,50664642,50743212,87,12,1492085,1.232000,0.108970,0.223740,SHANK3,Shank3
18862,49,ACR,22,50728224,50755299,13,3,1492085,1.734600,0.041403,0.112826,ACR,Acr


In [117]:
ext_bonf=set(ext_magma[ext_magma['P']<ext_bonf_cut]['Gene1Symbol'])

In [124]:
ext_magma=ext_magma[~ext_magma.Gene1Symbol.isna()]

In [43]:
ext_magma.to_csv('magma/seed_genes/ext_orig_annot_rat_ortho.tsv',sep='\t')

In [128]:
ext_magma

,ID,GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P,Q,Gene2Symbol,Gene1Symbol
1,148398,SAMD11,1,915741,954581,60,6,1492085,-0.920460,0.821330,0.864259,SAMD11,Samd11
2,26155,NOC2L,1,934203,969299,23,2,1492085,0.053753,0.478570,0.603460,NOC2L,Noc2l
3,339451,KLHL17,1,949952,975720,10,1,1492085,0.542700,0.293670,0.437381,KLHL17,Klhl17
4,84069,PLEKHN1,1,956497,985108,3,1,1492085,1.918900,0.027500,0.084303,PLEKHN1,Plekhn1
5,84808,PERM1,1,965199,992117,1,1,1492085,2.181400,0.014578,0.052566,PERM1,Perm1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18858,23542,MAPK8IP2,22,50590685,50621551,98,18,1492085,3.047000,0.001156,0.007306,MAPK8IP2,Mapk8ip2
18860,410,ARSA,22,50612754,50638173,74,12,1492085,1.106300,0.134300,0.257876,ARSA,Arsa
18861,85358,SHANK3,22,50664642,50743212,87,12,1492085,1.232000,0.108970,0.223740,SHANK3,Shank3
18862,49,ACR,22,50728224,50755299,13,3,1492085,1.734600,0.041403,0.112826,ACR,Acr
